# Exercises Sheet 4 Search - Genetic Algorithms

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time

#random.seed(2)

class Game:
    def __init__(self, str_size):
        if str_size <= 0:
            raise ValueError("str_size must be a positive integer")
        self.str_size = str_size
        self.target_pattern = [random.choice((0,1)) for x in range(0,self.str_size)]

    def change_game_pattern(self):
        self.target_pattern = [random.choice((0,1)) for x in range(0,self.str_size)]

    def generate_random_bit_pattern(self):
        return [random.choice((0,1)) for x in range(0,self.str_size)]

    def is_correct_pattern(self, pattern):
        if len(pattern) != self.str_size:
            raise ValueError("Pattern size does not match the expected size")
        return pattern == self.target_pattern

    def calc_fitness(self, atual):
        if len(atual) != self.str_size:
            raise ValueError("Pattern size does not match the expected size")
        valor = 0
        for i in range(0,len(self.target_pattern)):
            if self.target_pattern[i] == atual[i]:
                valor += 1/len(self.target_pattern)
        
        return valor

    def mutate_pattern(self, pattern):
        if len(pattern) != self.str_size:
            print(pattern)
            print(self.target_pattern)
            raise ValueError("Pattern size does not match the expected size")

        position = random.randint(0, self.str_size - 1) # random position
        pattern[position] = 1 if pattern[position] == 0 else 0

        return pattern

def measure_performance_random_picks(str_size, num_trials):
    attempts = []
    times = []
    fitness_lista = []
    padrao_lista = []

    for nada in range(num_trials):
        game = Game(str_size)
        start_time = time.time()
        attempt_count = 0

        while True:
            pattern = game.generate_random_bit_pattern()
            fitness = game.calc_fitness(pattern)
            fitness_lista.append(fitness)
            attempt_count += 1
            if game.is_correct_pattern(pattern):
                break

        #print(fitness_lista)
        end_time = time.time()
        elapsed_time = end_time - start_time
        attempts.append(attempt_count)
        times.append(elapsed_time)

    return attempts, times

def only_better_fittness(init_str, max_iterations=1000): # Ex1 b)
    game = Game(len(init_str))
    current_pattern = init_str
    current_fitness = game.fitness(current_pattern)

    for iteration in range(max_iterations):
        mutated_pattern = game.mutate_pattern(current_pattern)
        mutated_fitness = game.fitness(mutated_pattern)

        if mutated_fitness > current_fitness:
            current_pattern = mutated_pattern
            current_fitness = mutated_fitness

        if game.is_correct_pattern(current_pattern):
            break

    return current_pattern, iteration + 1

def plot_performance(str_sizes, num_trials):

    attempts_data = []
    times_data = []

    for str_size in str_sizes:
        attempts, times = measure_performance_random_picks(str_size, num_trials)
        attempts_data.append(attempts)
        times_data.append(times)

    fig, axs = plt.subplots(2, 1, figsize=(10, 10))

    # Plot attempts
    axs[0].boxplot(attempts_data, positions=str_sizes, widths=0.6)
    axs[0].set_xlabel('Number of Bits')
    axs[0].set_ylabel('Number of Attempts')
    axs[0].set_title('Attempts vs Number of Bits')
    axs[0].set_xticks(str_sizes)

    # Plot times
    axs[1].boxplot(times_data, positions=str_sizes, widths=0.6)
    axs[1].set_xlabel('Number of Bits')
    axs[1].set_ylabel('Time (seconds)')
    axs[1].set_title('Time vs Number of Bits')
    axs[1].set_xticks(str_sizes)

    plt.tight_layout()

#print(f"(num_attempts, time) for str_size=4: {measure_performance_random_picks(str_size=4, num_trials=1)}")
#print(f"(num_attempts, time) for str_size=6: {measure_performance_random_picks(str_size=6, num_trials=1)}")
#print(f"(num_attempts, time) for str_size=8: {measure_performance_random_picks(str_size=8, num_trials=1)}")

# Example usage:
#str_size = 10
#init_str = game.generate_random_bit_pattern()
#final_pattern, iterations = only_better_fitness(init_str)
#print("Initial Pattern:", init_str)
#print("Final Pattern:", final_pattern)
#print("Iterations:", iterations)

# Define the pattern sizes to test#
#str_sizes = [2, 4, 8, 12, 16]#
# Plot the performance
#plot_performance(str_sizes, 30)

#myGame = Game(6)
#myPattern = myGame.generate_random_bit_pattern()
#print(myPattern)
#print(myGame.mutate_pattern(myPattern))

'''random.seed(2)
inicio = time.time()
game = Game(16)
solucao = game.generate_random_bit_pattern()
top30 = [[]]

for i in range(0,100):
    numero = game.generate_random_bit_pattern()
    top30.append((numero,game.calc_fitness(numero)))

top30.pop(0)
top30.sort(key=lambda x: x[1], reverse=True)

iteracoes = 0

while(top30[0][1]!=1 or top30[0][1] != top30[1][1]):
    iteracoes +=1
    top30 = top30[0:30]    

    for i in range(0,70):
        numero = game.mutate_pattern(top30[random.randint(0,29)][0])
        top30.append((numero,game.calc_fitness(numero)))
    top30.sort(key=lambda x: x[1], reverse=True)
fim = time.time()

print(fim - inicio)
print(top30)
print(iteracoes)'''


random.seed(2)
inicio = time.time()
game = Game(16)
solucao = game.generate_random_bit_pattern()
top30 = [[]]

for i in range(0,100):
    numero = game.generate_random_bit_pattern()
    top30.append((numero,game.calc_fitness(numero)))

top30.pop(0)
top30.sort(key=lambda x: x[1], reverse=True)

iteracoes = 0

while(top30[0][1]!=1 or top30[0][1] != top30[1][1]):
    iteracoes +=1
    top30 = top30[0:30]

    for i in range(0,70):
        rand = random.randint(0,29)
        numero = top30[rand][0][0:int(len(top30[0][0])/2)] + top30[rand][0][int(len(top30[0][0])/2):]
        top30.append((numero,game.calc_fitness(numero)))
    top30.sort(key=lambda x: x[1], reverse=True)

    print(top30)

#print(top30)


[([1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0], 0.8125), ([1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0], 0.8125), ([1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0], 0.8125), ([1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0], 0.8125), ([0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1], 0.75), ([0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0], 0.75), ([0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1], 0.75), ([0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0], 0.75), ([0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0], 0.75), ([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1], 0.6875), ([0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0], 0.6875), ([0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1], 0.6875), ([0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0], 0.6875), ([0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0], 0.6875), ([0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1], 0.6875), ([0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0], 0.6875), ([0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0]